In [7]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import cross_val_score

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.decomposition import SparsePCA

path_name = 'your_path_name_here/mental_health.csv'
df=pd.read_csv(path_name)
print(df)

df = df.drop_duplicates()

#REDUCED SAMPLING
import random
random.seed(1)
size = list(df.index)
sample = random.sample(size,1000)
df = df.loc[sample]

print(sum(df["label"]))

#Split the matrix into predictors and response 
msg=df["text"]
msg=msg.str.replace('[^a-zA-Z0-9]+'," h", regex = True) 
y=df['label'].values
y 


#Stemming and Tokenising
stemmer=PorterStemmer()
msg=msg.apply(lambda line:[stemmer.stem(token.lower()) for token in word_tokenize(line)]).apply(lambda token:" ".join(token))
msg=msg.apply(lambda line:[token for token in word_tokenize(line) if len(token)>2]).apply(lambda y:" ".join(y))

#Vectorising
tf=TfidfVectorizer() 
data_vec=tf.fit_transform(msg)
print(data_vec.shape)

#Train, Validation, and Test Split ###RANDOMSTATE=1###
from sklearn.model_selection import train_test_split
#test_size=0.3 implies 70% will be used to training and 30% for testing. 
#random_state sets seed for random number generator, ensuring reducibility
x_train,x_test,y_train,y_test=train_test_split(data_vec,y,test_size=0.3,random_state=1)

                                                    text  label
0      dear american teens question dutch person hear...      0
1      nothing look forward lifei dont many reasons k...      1
2      music recommendations im looking expand playli...      0
3      im done trying feel betterthe reason im still ...      1
4      worried  year old girl subject domestic physic...      1
...                                                  ...    ...
27972  posting everyday people stop caring  religion ...      0
27973  okay definetly need hear guys opinion ive pret...      0
27974  cant get dog think ill kill myselfthe last thi...      1
27975  whats point princess bridei really think like ...      1
27976  got nudes person might might know snapchat do ...      0

[27977 rows x 2 columns]
513
(1000, 7779)


In [8]:
x_train_dense = x_train.todense()
x_test_dense = x_test.todense()

In [9]:
#Sparse PCA #2 Components

pca = SparsePCA(n_components = 2)
x_train_pca = pca.fit_transform(x_train_dense)

modelLR = LogisticRegression(penalty='none')

modelLR.fit(x_train_pca,y_train)

# Transform the test set as well 
x_test_pca = pca.transform(x_test_dense)

# Make predictions on a new set of data
predictions = modelLR.predict(x_test_pca)

# Compare the predicted labels to the true labels
accuracy_on_test_set = modelLR.score(x_test_pca, y_test) #accuracy = # of correct predictions / total # of predictions
print("Accuracy on the test set:", accuracy_on_test_set)
print(classification_report(y_test,predictions))

/Users/tobyokeefe/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


/Users/tobyokeefe/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Accuracy on the test set: 0.6833333333333333
              precision    recall  f1-score   support

           0       0.64      0.84      0.73       152
           1       0.76      0.53      0.62       148

    accuracy                           0.68       300
   macro avg       0.70      0.68      0.67       300
weighted avg       0.70      0.68      0.68       300



/Users/tobyokeefe/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [10]:
#Logistic Lasso with 5 fold Cross Validation on Sparse PCA reduced data (2 components)

alphas = 10.0 ** np.arange(-5, 5, 0.1)

cv_scores = [] 

# Perform cross-validation for each value of C
for alpha in alphas:
    modelLR_lasso = LogisticRegression(penalty='l1', C = 1/alpha, random_state=1, solver='liblinear')
    scores = cross_val_score(modelLR_lasso, x_train_pca, y_train, cv = 5)  # 5-fold cross-validation
    cv_scores.append(np.mean(scores))


# Find the index corresponding to the best C value
best_CV_index = np.argmax(cv_scores)
best_alpha = alphas[best_CV_index]

# Print or inspect the best C value
print("Best C value selected by cross-validation:", 1/best_alpha)
print("Best lambda value (regularisation value) selected by cross-validation:", best_alpha)

modelLR_lasso = LogisticRegression(penalty='l1', C=1/best_alpha, random_state=1, solver = 'liblinear')

modelLR_lasso.fit(x_train_pca,y_train)


# Make predictions on a new set of data
predictions = modelLR_lasso.predict(x_test_pca)

# Compare the predicted labels to the true labels
accuracy_on_test_set = modelLR_lasso.score(x_test_pca, y_test) #accuaracy = # of correct predictions / total # of predictions

print("Accuracy on the test set:", accuracy_on_test_set)

print(classification_report(y_test,predictions))

Best C value selected by cross-validation: 0.2511886431509695
Best lambda value (regularisation value) selected by cross-validation: 3.98107170553479
Accuracy on the test set: 0.6866666666666666
              precision    recall  f1-score   support

           0       0.65      0.85      0.73       152
           1       0.77      0.52      0.62       148

    accuracy                           0.69       300
   macro avg       0.71      0.68      0.68       300
weighted avg       0.71      0.69      0.68       300



In [11]:
#Logistic Ridge with 5 fold Cross Validation on PCA reduced data (2 components)

alphas = 10.0 ** np.arange(-5, 5, 0.1)

cv_scores = []  

# Perform cross-validation for each value of C
for alpha in alphas:
    modelLR_Ridge = LogisticRegression(penalty='l2', C = 1/alpha, random_state=1, solver='liblinear')
    scores = cross_val_score(modelLR_Ridge, x_train_pca, y_train, cv = 5)  # 5-fold cross-validation
    cv_scores.append(np.mean(scores))


# Find the index corresponding to the best C value
best_CV_index = np.argmax(cv_scores)
best_alpha = alphas[best_CV_index]

# Print or inspect the best C value
print("Best C value selected by cross-validation:", 1/best_alpha)
print("Best lambda value (regularisation value) selected by cross-validation:", best_alpha)

modelLR_Ridge = LogisticRegression(penalty='l2', C=1/best_alpha, random_state=1, solver = 'liblinear')

modelLR_Ridge.fit(x_train_pca,y_train)


# Make predictions on a new set of data
predictions = modelLR_Ridge.predict(x_test_pca)

# Compare the predicted labels to the true labels
accuracy_on_test_set = modelLR_Ridge.score(x_test_pca, y_test) #accuaracy = # of correct predictions / total # of predictions

print("Accuracy on the test set:", accuracy_on_test_set)

print(classification_report(y_test,predictions))

Best C value selected by cross-validation: 0.2511886431509695
Best lambda value (regularisation value) selected by cross-validation: 3.98107170553479
Accuracy on the test set: 0.7
              precision    recall  f1-score   support

           0       0.70      0.72      0.71       152
           1       0.70      0.68      0.69       148

    accuracy                           0.70       300
   macro avg       0.70      0.70      0.70       300
weighted avg       0.70      0.70      0.70       300



In [12]:
#PCA #4 Components

pca = SparsePCA(n_components = 4)
x_train_pca = pca.fit_transform(x_train_dense)

modelLR = LogisticRegression(penalty='none')

modelLR.fit(x_train_pca,y_train)

# Transform the test set as well 
x_test_pca = pca.transform(x_test_dense)

# Make predictions on a new set of data
predictions = modelLR.predict(x_test_pca)

# Compare the predicted labels to the true labels
accuracy_on_test_set = modelLR.score(x_test_pca, y_test) #accuracy = # of correct predictions / total # of predictions
print("Accuracy on the test set:", accuracy_on_test_set)
print(classification_report(y_test,predictions))

/Users/tobyokeefe/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Accuracy on the test set: 0.69
              precision    recall  f1-score   support

           0       0.65      0.83      0.73       152
           1       0.76      0.55      0.64       148

    accuracy                           0.69       300
   macro avg       0.70      0.69      0.68       300
weighted avg       0.70      0.69      0.68       300



/Users/tobyokeefe/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/Users/tobyokeefe/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [13]:
#Logistic Lasso with 5 fold Cross Validation on Sparse PCA reduced data (4 components)

alphas = 10.0 ** np.arange(-5, 5, 0.1)

cv_scores = [] 

# Perform cross-validation for each value of C
for alpha in alphas:
    modelLR_lasso = LogisticRegression(penalty='l1', C = 1/alpha, random_state=1, solver='liblinear')
    scores = cross_val_score(modelLR_lasso, x_train_pca, y_train, cv = 5)  # 5-fold cross-validation
    cv_scores.append(np.mean(scores))

#CV scores by default are the score method of the estimator, in this case the logistic lasso the score is the mean accuracy of self predict on the data

# Find the index corresponding to the best C value
best_CV_index = np.argmax(cv_scores)
best_alpha = alphas[best_CV_index]

# Print or inspect the best C value
print("Best C value selected by cross-validation:", 1/best_alpha)
print("Best lambda value (regularisation value) selected by cross-validation:", best_alpha)

modelLR_lasso = LogisticRegression(penalty='l1', C=1/best_alpha, random_state=1, solver = 'liblinear')

modelLR_lasso.fit(x_train_pca,y_train)


# Make predictions on a new set of data
predictions = modelLR_lasso.predict(x_test_pca)

# Compare the predicted labels to the true labels
accuracy_on_test_set = modelLR_lasso.score(x_test_pca, y_test) #accuaracy = # of correct predictions / total # of predictions

print("Accuracy on the test set:", accuracy_on_test_set)

print(classification_report(y_test,predictions))

Best C value selected by cross-validation: 6.309573444802149
Best lambda value (regularisation value) selected by cross-validation: 0.1584893192461059
Accuracy on the test set: 0.6933333333333334
              precision    recall  f1-score   support

           0       0.65      0.84      0.73       152
           1       0.76      0.55      0.64       148

    accuracy                           0.69       300
   macro avg       0.71      0.69      0.69       300
weighted avg       0.71      0.69      0.69       300



In [14]:
#Logistic Ridge with 5 fold Cross Validation on PCA reduced data (4 components)

alphas = 10.0 ** np.arange(-5, 5, 0.1)

cv_scores = [] ###CAN'T USE LAMBDA SINCE ITS A COMMAND 

# Perform cross-validation for each value of C
for alpha in alphas:
    modelLR_Ridge = LogisticRegression(penalty='l2', C = 1/alpha, random_state=1, solver='liblinear')
    scores = cross_val_score(modelLR_Ridge, x_train_pca, y_train, cv = 5)  # 5-fold cross-validation
    cv_scores.append(np.mean(scores))

#CV scores by default are the score method of the estimator, in this case the logistic lasso the score is the mean accuracy of self predict on the data

# Find the index corresponding to the best C value
best_CV_index = np.argmax(cv_scores)
best_alpha = alphas[best_CV_index]

# Print or inspect the best C value
print("Best C value selected by cross-validation:", 1/best_alpha)
print("Best lambda value (regularisation value) selected by cross-validation:", best_alpha)

modelLR_Ridge = LogisticRegression(penalty='l2', C=1/best_alpha, random_state=1, solver = 'liblinear')

modelLR_Ridge.fit(x_train_pca,y_train)


# Make predictions on a new set of data
predictions = modelLR_Ridge.predict(x_test_pca)

# Compare the predicted labels to the true labels
accuracy_on_test_set = modelLR_Ridge.score(x_test_pca, y_test) #accuaracy = # of correct predictions / total # of predictions

print("Accuracy on the test set:", accuracy_on_test_set)

print(classification_report(y_test,predictions))

Best C value selected by cross-validation: 63.09573444802097
Best lambda value (regularisation value) selected by cross-validation: 0.01584893192461072
Accuracy on the test set: 0.69
              precision    recall  f1-score   support

           0       0.65      0.83      0.73       152
           1       0.76      0.55      0.64       148

    accuracy                           0.69       300
   macro avg       0.70      0.69      0.68       300
weighted avg       0.70      0.69      0.68       300



In [15]:
#PCA #6 Components

pca = SparsePCA(n_components = 6)
x_train_pca = pca.fit_transform(x_train_dense)

modelLR = LogisticRegression(penalty='none')

modelLR.fit(x_train_pca,y_train)

# Transform the test set as well 
x_test_pca = pca.transform(x_test_dense)

# Make predictions on a new set of data
predictions = modelLR.predict(x_test_pca)

# Compare the predicted labels to the true labels
accuracy_on_test_set = modelLR.score(x_test_pca, y_test) #accuracy = # of correct predictions / total # of predictions
print("Accuracy on the test set:", accuracy_on_test_set)
print(classification_report(y_test,predictions))

/Users/tobyokeefe/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Accuracy on the test set: 0.6866666666666666
              precision    recall  f1-score   support

           0       0.65      0.83      0.73       152
           1       0.75      0.54      0.63       148

    accuracy                           0.69       300
   macro avg       0.70      0.68      0.68       300
weighted avg       0.70      0.69      0.68       300



/Users/tobyokeefe/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/Users/tobyokeefe/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [16]:
#Logistic Lasso with 5 fold Cross Validation on Sparse PCA reduced data (6 components)

alphas = 10.0 ** np.arange(-5, 5, 0.1)

cv_scores = [] 

# Perform cross-validation for each value of C
for alpha in alphas:
    modelLR_lasso = LogisticRegression(penalty='l1', C = 1/alpha, random_state=1, solver='liblinear')
    scores = cross_val_score(modelLR_lasso, x_train_pca, y_train, cv = 5)  # 5-fold cross-validation
    cv_scores.append(np.mean(scores))

#CV scores by default are the score method of the estimator, in this case the logistic lasso the score is the mean accuracy of self predict on the data

# Find the index corresponding to the best C value
best_CV_index = np.argmax(cv_scores)
best_alpha = alphas[best_CV_index]

# Print or inspect the best C value
print("Best C value selected by cross-validation:", 1/best_alpha)
print("Best lambda value (regularisation value) selected by cross-validation:", best_alpha)

modelLR_lasso = LogisticRegression(penalty='l1', C=1/best_alpha, random_state=1, solver = 'liblinear')

modelLR_lasso.fit(x_train_pca,y_train)


# Make predictions on a new set of data
predictions = modelLR_lasso.predict(x_test_pca)

# Compare the predicted labels to the true labels
accuracy_on_test_set = modelLR_lasso.score(x_test_pca, y_test) #accuaracy = # of correct predictions / total # of predictions

print("Accuracy on the test set:", accuracy_on_test_set)

print(classification_report(y_test,predictions))

Best C value selected by cross-validation: 3.162277660168496
Best lambda value (regularisation value) selected by cross-validation: 0.3162277660168263
Accuracy on the test set: 0.69
              precision    recall  f1-score   support

           0       0.65      0.84      0.73       152
           1       0.76      0.54      0.63       148

    accuracy                           0.69       300
   macro avg       0.71      0.69      0.68       300
weighted avg       0.71      0.69      0.68       300



In [17]:
#Logistic Ridge with 5 fold Cross Validation on PCA reduced data (6 components)

alphas = 10.0 ** np.arange(-5, 5, 0.1)

cv_scores = [] ###CAN'T USE LAMBDA SINCE ITS A COMMAND 

# Perform cross-validation for each value of C
for alpha in alphas:
    modelLR_Ridge = LogisticRegression(penalty='l2', C = 1/alpha, random_state=1, solver='liblinear')
    scores = cross_val_score(modelLR_Ridge, x_train_pca, y_train, cv = 5)  # 5-fold cross-validation
    cv_scores.append(np.mean(scores))

#CV scores by default are the score method of the estimator, in this case the logistic lasso the score is the mean accuracy of self predict on the data

# Find the index corresponding to the best C value
best_CV_index = np.argmax(cv_scores)
best_alpha = alphas[best_CV_index]

# Print or inspect the best C value
print("Best C value selected by cross-validation:", 1/best_alpha)
print("Best lambda value (regularisation value) selected by cross-validation:", best_alpha)

modelLR_Ridge = LogisticRegression(penalty='l2', C=1/best_alpha, random_state=1, solver = 'liblinear')

modelLR_Ridge.fit(x_train_pca,y_train)


# Make predictions on a new set of data
predictions = modelLR_Ridge.predict(x_test_pca)

# Compare the predicted labels to the true labels
accuracy_on_test_set = modelLR_Ridge.score(x_test_pca, y_test) #accuaracy = # of correct predictions / total # of predictions

print("Accuracy on the test set:", accuracy_on_test_set)

print(classification_report(y_test,predictions))

Best C value selected by cross-validation: 99999.99999999999
Best lambda value (regularisation value) selected by cross-validation: 1e-05
Accuracy on the test set: 0.6866666666666666
              precision    recall  f1-score   support

           0       0.65      0.83      0.73       152
           1       0.75      0.54      0.63       148

    accuracy                           0.69       300
   macro avg       0.70      0.68      0.68       300
weighted avg       0.70      0.69      0.68       300

